# Redis Workshop 2026 - Google Colab Edition

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_REPO/redis_workshop_colab.ipynb)

Este notebook contém exercícios práticos com Redis mostrando **comandos redis-cli** e **código Python** lado a lado.

## 🚀 Vantagens desta versão:
- ✅ **Zero instalação** - Roda direto no navegador
- ✅ **Instalação automática** de dependências
- ✅ **Comandos redis-cli puros** - Aprenda a sintaxe original
- ✅ **Código Python equivalente** - Veja como usar em aplicações

## 📋 Pré-requisitos

Apenas uma conta no **Redis Cloud** (Free Tier):
1. Acesse: https://redis.com/try-free/
2. Crie um database Free Tier
3. Copie as credenciais (host, port, password)

## 🔧 Instalação Automática de Dependências

Execute a célula abaixo para instalar o redis-py:

In [ ]:
# Instalar redis-py
!pip install redis -q
print("✅ Dependências instaladas com sucesso!")

## 🔌 Configuração da Conexão

Configure suas credenciais do Redis Cloud:

In [ ]:
import redis
import json

# ⚠️ CONFIGURE SUAS CREDENCIAIS AQUI
REDIS_HOST = "your-redis-host.redis.cloud"  # Ex: redis-12345.c123.us-east-1-1.ec2.cloud.redislabs.com
REDIS_PORT = 12345  # Sua porta
REDIS_PASSWORD = "your-password"  # Sua senha

# Criar conexão
r = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD,
    decode_responses=True
)

# Testar conexão
try:
    r.ping()
    print("✅ Conexão com Redis estabelecida com sucesso!")
    print(f"📍 Conectado em: {REDIS_HOST}:{REDIS_PORT}")
except Exception as e:
    print(f"❌ Erro ao conectar: {e}")
    print("\n💡 Verifique se:")
    print("   1. As credenciais estão corretas")
    print("   2. O database está ativo no Redis Cloud")
    print("   3. Seu IP está permitido no firewall (Redis Cloud permite todos por padrão)")

---
# 📖 Como Usar Este Notebook

Cada exercício mostra:

### 1️⃣ Comandos redis-cli
```redis
SET key "value"
GET key
```

### 2️⃣ Código Python equivalente
```python
r.set("key", "value")
r.get("key")
```

### 3️⃣ Resultado esperado
```
Output: value
```

---
# 1. String

Strings são o tipo de dado mais básico do Redis. Ideais para cache, tokens, contadores.

## Exercício 1.1: Cache de Dados de Mercado

### 📝 Comandos redis-cli:
```redis
# Cache de cotação em tempo real
SET price:PETR4 "34.22"
SET price:VALE3 "72.10"

# Leitura rápida para telas e APIs
GET price:PETR4
GET price:VALE3

# Atualizar preço
SET price:PETR4 "34.35"
```

### 🐍 Código Python:

In [ ]:
# Cache de cotação em tempo real
r.set("price:PETR4", "34.22")
r.set("price:VALE3", "72.10")
print("✅ Preços salvos no cache")

# Leitura rápida para telas e APIs
petr4_price = r.get("price:PETR4")
vale3_price = r.get("price:VALE3")

print(f"\n📊 Cotações:")
print(f"   PETR4: R$ {petr4_price}")
print(f"   VALE3: R$ {vale3_price}")

# Atualizar preço
r.set("price:PETR4", "34.35")
print(f"\n🔄 Preço atualizado PETR4: R$ {r.get('price:PETR4')}")

## Exercício 1.2: Tokens de Autenticação

### 📝 Comandos redis-cli:
```redis
# Salvar token JWT com expiração de 15 minutos (900 segundos)
SETEX auth:token:xyz123 900 "jwt_header.jwt_payload.jwt_signature"

# Ler token na validação
GET auth:token:xyz123

# Verificar tempo restante
TTL auth:token:xyz123

# Invalidar na hora (logout)
DEL auth:token:xyz123
```

### 🐍 Código Python:

In [ ]:
# Salvar token JWT com expiração de 15 minutos (900 segundos)
r.setex("auth:token:xyz123", 900, "jwt_header.jwt_payload.jwt_signature")
print("✅ Token salvo com expiração de 15 minutos")

# Ler token na validação
token = r.get("auth:token:xyz123")
print(f"\n🔑 Token: {token[:30]}...")

# Verificar TTL (Time To Live)
ttl = r.ttl("auth:token:xyz123")
print(f"⏱️  Tempo restante: {ttl} segundos ({ttl//60} minutos)")

# Invalidar na hora (logout)
r.delete("auth:token:xyz123")
print(f"\n🚪 Token após logout: {r.get('auth:token:xyz123')}")

---
# 2. Hash

Hashes são mapas de campos e valores. Ideais para representar objetos.

## Exercício 2.1: Perfis de Cliente

### 📝 Comandos redis-cli:
```redis
# Criar/atualizar perfil
HSET customer:1001 name "Ana XP" segment "alta_renda" risk "moderado"

# Ler atributo específico
HGET customer:1001 segment

# Verificar se campo existe
HEXISTS customer:1001 advisor_id

# Adicionar campo novo
HSET customer:1001 advisor_id "A-77"

# Listar todos os campos
HGETALL customer:1001
```

### 🐍 Código Python:

In [ ]:
# Criar/atualizar perfil
r.hset("customer:1001", mapping={
    "name": "Ana XP",
    "segment": "alta_renda",
    "risk": "moderado"
})
print("✅ Perfil criado")

# Ler atributo específico
segment = r.hget("customer:1001", "segment")
print(f"\n👤 Segmento: {segment}")

# Verificar se campo existe
has_advisor = r.hexists("customer:1001", "advisor_id")
print(f"📋 Tem advisor_id? {bool(has_advisor)}")

# Adicionar campo novo
r.hset("customer:1001", "advisor_id", "A-77")
print("✅ Advisor adicionado")

# Listar todos os campos
customer_data = r.hgetall("customer:1001")
print(f"\n📊 Dados completos do cliente:")
for key, value in customer_data.items():
    print(f"   {key}: {value}")

## Exercício 2.2: Saldos e Limites

### 📝 Comandos redis-cli:
```redis
# Criar registro de limites e saldo
HSET account:1001 balance 2500.00 credit_limit 10000 available_limit 7500

# Débito por compra aprovada
HINCRBYFLOAT account:1001 balance -500.00
HINCRBYFLOAT account:1001 available_limit -500.00

# Crédito após pagamento
HINCRBYFLOAT account:1001 balance 500.00
HINCRBYFLOAT account:1001 available_limit 500.00

# Consultar saldo e limites
HGETALL account:1001
```

### 🐍 Código Python:

In [ ]:
# Criar registro de limites e saldo
r.hset("account:1001", mapping={
    "balance": "2500.00",
    "credit_limit": "10000",
    "available_limit": "7500"
})
print("✅ Conta criada")

# Débito por compra aprovada
new_balance = r.hincrbyfloat("account:1001", "balance", -500.00)
new_limit = r.hincrbyfloat("account:1001", "available_limit", -500.00)

print(f"\n💳 Após débito de R$ 500:")
print(f"   Saldo: R$ {new_balance:.2f}")
print(f"   Limite disponível: R$ {new_limit:.2f}")

# Crédito após pagamento
r.hincrbyfloat("account:1001", "balance", 500.00)
r.hincrbyfloat("account:1001", "available_limit", 500.00)

# Consultar saldo e limites
account_info = r.hgetall("account:1001")
print(f"\n💰 Após crédito de R$ 500:")
for key, value in account_info.items():
    print(f"   {key}: R$ {value}")

---
# 3. Sets

Sets são coleções não ordenadas de strings únicas. Ideais para tags, membros, listas.

## Exercício 3.1: Lista de Monitoramento de Fraude

### 📝 Comandos redis-cli:
```redis
# Adicionar clientes à lista
SADD fraud:watchlist 1001 2002 3003

# Verificar se cliente está na lista
SISMEMBER fraud:watchlist 2002

# Remover cliente
SREM fraud:watchlist 3003

# Listar todos
SMEMBERS fraud:watchlist
```

### 🐍 Código Python:

In [ ]:
# Adicionar clientes à lista de monitoramento
r.sadd("fraud:watchlist", 1001, 2002, 3003)
print("✅ Clientes adicionados à watchlist")

# Verificar se cliente está na lista
is_monitored = r.sismember("fraud:watchlist", 2002)
print(f"\n🔍 Cliente 2002 está monitorado? {bool(is_monitored)}")

# Remover cliente
r.srem("fraud:watchlist", 3003)
print("✅ Cliente 3003 removido")

# Listar todos os clientes monitorados
monitored = r.smembers("fraud:watchlist")
print(f"\n📋 Clientes monitorados: {monitored}")

---
# 4. Sorted Sets

Sorted Sets são sets ordenados por um score. Ideais para rankings e leaderboards.

## Exercício 4.1: Ranking de Ativos Mais Negociados

### 📝 Comandos redis-cli:
```redis
# Incrementar volume negociado
ZINCRBY ranking:ativos 100000 "PETR4"
ZINCRBY ranking:ativos 80000 "VALE3"
ZINCRBY ranking:ativos 50000 "ITUB4"

# Ranking do mais negociado para o menos
ZREVRANGE ranking:ativos 0 -1 WITHSCORES

# Posição de um ativo no ranking
ZREVRANK ranking:ativos "PETR4"
```

### 🐍 Código Python:

In [ ]:
# Incrementar volume negociado
r.zincrby("ranking:ativos", 100000, "PETR4")
r.zincrby("ranking:ativos", 80000, "VALE3")
r.zincrby("ranking:ativos", 50000, "ITUB4")
print("✅ Volumes registrados")

# Ranking do mais negociado para o menos
ranking = r.zrevrange("ranking:ativos", 0, -1, withscores=True)
print(f"\n🏆 Ranking de ativos mais negociados:")
for i, (ticker, volume) in enumerate(ranking, 1):
    print(f"   {i}º {ticker}: R$ {volume:,.0f}")

# Posição de um ativo no ranking
position = r.zrevrank("ranking:ativos", "PETR4")
print(f"\n📊 Posição de PETR4: {position + 1}º lugar")

---
# 5. Lists

Lists são listas ordenadas de strings. Ideais para filas e históricos.

## Exercício 5.1: Fila de Processamento de Ordens

### 📝 Comandos redis-cli:
```redis
# Enfileirar novas ordens (FIFO)
RPUSH orders:queue "order:1" "order:2" "order:3"

# Ver fila atual
LRANGE orders:queue 0 -1

# Consumir próxima ordem
LPOP orders:queue

# Consumir com bloqueio (aguarda até 5 min)
BLPOP orders:queue 300
```

### 🐍 Código Python:

In [ ]:
# Enfileirar novas ordens (FIFO)
r.rpush("orders:queue", "order:1", "order:2", "order:3")
print("✅ Ordens enfileiradas")

# Ver fila atual
queue = r.lrange("orders:queue", 0, -1)
print(f"\n📋 Fila atual: {queue}")

# Consumir próxima ordem
next_order = r.lpop("orders:queue")
print(f"\n⚙️  Processando: {next_order}")

# Ver fila após consumo
queue = r.lrange("orders:queue", 0, -1)
print(f"📋 Fila após consumo: {queue}")

# Nota: BLPOP é bloqueante
# result = r.blpop("orders:queue", timeout=300)
# print(f"Ordem consumida (bloqueante): {result}")

---
# 6. Streams

Streams são logs append-only com consumer groups. Ideais para event sourcing.

## Exercício 6.1: Pipeline de Pagamentos com Consumer Groups

### 📝 Comandos redis-cli:
```redis
# Criar grupos
XGROUP CREATE payments fraud 0 MKSTREAM
XGROUP CREATE payments settlement 0

# Adicionar pagamentos
XADD payments * paymentId "pmt-1" customerId "1001" amount "500.00" method "PIX"
XADD payments * paymentId "pmt-2" customerId "1002" amount "3500.00" method "PIX"

# Ler no serviço de fraude
XREADGROUP GROUP fraud worker-f1 COUNT 2 STREAMS payments >

# Confirmar processamento
XACK payments fraud <message-id>
```

### 🐍 Código Python:

In [ ]:
# Criar os grupos
try:
    r.xgroup_create("payments", "fraud", id="0", mkstream=True)
    r.xgroup_create("payments", "settlement", id="0")
    print("✅ Grupos criados")
except redis.ResponseError:
    print("⚠️  Grupos já existem")

# Adicionar pagamentos
pmt1 = r.xadd("payments", {
    "paymentId": "pmt-1",
    "customerId": "1001",
    "amount": "500.00",
    "method": "PIX"
})

pmt2 = r.xadd("payments", {
    "paymentId": "pmt-2",
    "customerId": "1002",
    "amount": "3500.00",
    "method": "PIX"
})

print(f"\n💳 Pagamentos criados:")
print(f"   Payment 1: {pmt1}")
print(f"   Payment 2: {pmt2}")

# Ler no serviço de fraude
fraud_msgs = r.xreadgroup("fraud", "worker-f1", {"payments": ">"}, count=2)
print(f"\n🔍 Serviço de fraude processando:")
for stream_name, messages in fraud_msgs:
    for msg_id, msg_data in messages:
        print(f"   ID: {msg_id}")
        print(f"   Dados: {msg_data}")
        # Confirmar processamento
        r.xack("payments", "fraud", msg_id)
        print(f"   ✓ Confirmado\n")

---
# 7. HyperLogLog

HyperLogLog conta elementos únicos com uso mínimo de memória.

## Exercício 7.1: DAU/MAU - Usuários Únicos

### 📝 Comandos redis-cli:
```redis
# Registrar usuários por dia
PFADD xp:unique:2025-05-01 1001 1002 1003 1001
PFADD xp:unique:2025-05-02 1001 4004
PFADD xp:unique:2025-05-03 2002 5005

# DAU (Daily Active Users)
PFCOUNT xp:unique:2025-05-01

# Unificar o mês
PFMERGE xp:unique:2025-05 xp:unique:2025-05-01 xp:unique:2025-05-02 xp:unique:2025-05-03

# MAU (Monthly Active Users)
PFCOUNT xp:unique:2025-05
```

### 🐍 Código Python:

In [ ]:
# Registrar usuários por dia (duplicatas são ignoradas)
r.pfadd("xp:unique:2025-05-01", 1001, 1002, 1003, 1001)
r.pfadd("xp:unique:2025-05-02", 1001, 4004)
r.pfadd("xp:unique:2025-05-03", 2002, 5005)
print("✅ Usuários registrados")

# DAU (Daily Active Users)
dau = r.pfcount("xp:unique:2025-05-01")
print(f"\n📊 DAU 2025-05-01: {dau} usuários únicos")

# Unificar o mês
r.pfmerge("xp:unique:2025-05",
          "xp:unique:2025-05-01",
          "xp:unique:2025-05-02",
          "xp:unique:2025-05-03")

# MAU (Monthly Active Users)
mau = r.pfcount("xp:unique:2025-05")
print(f"📊 MAU 2025-05: {mau} usuários únicos")

---
# 🧹 Limpeza (Opcional)

Execute para limpar as chaves criadas durante o workshop.

In [ ]:
# Deletar chaves específicas do workshop
patterns = [
    "price:*", "auth:*", "customer:*", "account:*",
    "fraud:*", "ranking:*", "orders:*", "payments", "xp:*"
]

deleted_count = 0
for pattern in patterns:
    keys = r.keys(pattern)
    if keys:
        deleted_count += r.delete(*keys)

print(f"✅ {deleted_count} chaves removidas")

# CUIDADO: Para deletar TUDO (descomente se tiver certeza)
# r.flushdb()
# print("✅ Todas as chaves foram removidas")

---
# 🎉 Conclusão

Parabéns! Você completou o Redis Workshop 2026!

## 📚 O que você aprendeu:

1. **Strings** - Cache, tokens, contadores
2. **Hashes** - Perfis, saldos, objetos estruturados
3. **Sets** - Listas únicas, monitoramento
4. **Sorted Sets** - Rankings, leaderboards
5. **Lists** - Filas, históricos
6. **Streams** - Event sourcing, consumer groups
7. **HyperLogLog** - Contagem de únicos (DAU/MAU)

## 🚀 Próximos Passos:

- 📖 [Redis Documentation](https://redis.io/docs/)
- 🔍 [Redis Insight](https://redis.io/insight/) - Interface gráfica
- 📦 [RedisJSON](https://redis.io/docs/stack/json/) - Documentos JSON
- 🔄 [Redis Pub/Sub](https://redis.io/docs/manual/pubsub/) - Mensagens em tempo real
- 🌍 [Redis Geo](https://redis.io/docs/manual/data-types/geospatial/) - Dados geoespaciais
- 🎯 [Redis Bloom](https://redis.io/docs/stack/bloom/) - Filtros probabilísticos

## 💡 Dicas Finais:

1. **Use TTL** para dados temporários (cache, sessões)
2. **Escolha a estrutura certa** para cada caso de uso
3. **Monitore memória** - Redis é in-memory
4. **Use pipelines** para múltiplos comandos
5. **Considere Redis Cluster** para escalabilidade

## 🔗 Recursos Adicionais:

- [Redis University](https://university.redis.com/) - Cursos gratuitos
- [Redis Best Practices](https://redis.io/docs/manual/patterns/)
- [Redis CLI Reference](https://redis.io/docs/manual/cli/)

---

### 📝 Feedback

Gostou do workshop? Tem sugestões? Compartilhe seu feedback!

**Happy Caching! 🚀**